<a href="https://colab.research.google.com/github/UrayLaszlo/Airbnb_project/blob/main/nyc_airbnb_02_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import re, requests, pandas as pd, io, gzip, pathlib, os, datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
listings = pd.read_csv('listings.csv.gz')
calendar = pd.read_csv('calendar.csv.gz')
reviews = pd.read_csv('reviews.csv.gz')

listings, calendar, reviews

<ipython-input-6-abc17637b13e>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  calendar = pd.read_csv('calendar.csv.gz')


(                        id                                       listing_url  \
 0                    36121                https://www.airbnb.com/rooms/36121   
 1                    36647                https://www.airbnb.com/rooms/36647   
 2                    38663                https://www.airbnb.com/rooms/38663   
 3                    38833                https://www.airbnb.com/rooms/38833   
 4                    39282                https://www.airbnb.com/rooms/39282   
 ...                    ...                                               ...   
 37429  1366333532374850165  https://www.airbnb.com/rooms/1366333532374850165   
 37430  1366717321390111215  https://www.airbnb.com/rooms/1366717321390111215   
 37431  1366721904709517353  https://www.airbnb.com/rooms/1366721904709517353   
 37432  1366722692755341871  https://www.airbnb.com/rooms/1366722692755341871   
 37433  1366723228243064949  https://www.airbnb.com/rooms/1366723228243064949   
 
             scrape_id las

In [7]:
def classify_column(dtype, n_unique, pct_missing, n_rows):
    """Heuristic label for each column."""
    # IDs / constant-ish ----------------------------------------------------
    if n_unique == n_rows:                       # every row different
        return "id-like"
    if n_unique <= 1:                            # all the same
        return "constant"

    # Core data types -------------------------------------------------------
    if pd.api.types.is_bool_dtype(dtype):
        return "boolean"
    if pd.api.types.is_datetime64_any_dtype(dtype):
        return "datetime"
    if pd.api.types.is_numeric_dtype(dtype):
        # hand-wavy split between discrete and continuous
        return "numeric_discrete" if n_unique < 0.02 * n_rows else "numeric_continuous"
    # object, string, category, etc.
    return "categorical_low"  if n_unique < 20 else "categorical_high"

def column_summary(listings):
    """Return a DataFrame describing every column."""
    n_rows = len(listings)
    out = (
        pd.DataFrame({
            "dtype":       listings.dtypes,
            "n_unique":    listings.nunique(dropna=False),
            "pct_missing": listings.isna().mean().round(3) * 100,
        })
        .assign(
            category=lambda t: [
                classify_column(t.dtype[i], t.n_unique[i], t.pct_missing[i], n_rows)
                for i in range(len(t))
            ]
        )
        .sort_values("category")
    )
    return out

# --- run it on your data ---
summary = column_summary(listings)
summary.head(20)        # peek, or just display(summary) in Colab


<ipython-input-7-917472a4fda0>:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  classify_column(t.dtype[i], t.n_unique[i], t.pct_missing[i], n_rows)


,dtype,n_unique,pct_missing,category
amenities,object,30306,0.0,categorical_high
property_type,object,70,0.0,categorical_high
neighbourhood_cleansed,object,223,0.0,categorical_high
first_review,object,4338,31.5,categorical_high
last_review,object,3249,31.5,categorical_high
host_neighbourhood,object,517,20.6,categorical_high
host_picture_url,object,21626,0.1,categorical_high
host_thumbnail_url,object,21626,0.1,categorical_high
price,object,1023,40.4,categorical_high
host_acceptance_rate,object,99,39.3,categorical_high


In [8]:
summary

,dtype,n_unique,pct_missing,category
amenities,object,30306,0.0,categorical_high
property_type,object,70,0.0,categorical_high
neighbourhood_cleansed,object,223,0.0,categorical_high
first_review,object,4338,31.5,categorical_high
last_review,object,3249,31.5,categorical_high
...,...,...,...,...
availability_365,int64,366,0.0,numeric_discrete
number_of_reviews,int64,498,0.0,numeric_discrete
number_of_reviews_ltm,int64,173,0.0,numeric_discrete
minimum_nights_avg_ntm,float64,457,0.0,numeric_discrete


In [9]:
summary.describe()

,n_unique,pct_missing
count,79.000000,79.000000
mean,5088.936709,12.634177
std,10526.858655,20.805164
min,1.000000,0.000000
25%,24.500000,0.000000
50%,150.000000,0.000000
75%,1517.500000,31.500000
max,37434.000000,100.000000
